In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
#tf.enable_v2_behavior()
# Import tensornetwork
import tensornetwork as tn
# Set the backend to tesorflow
# (default is numpy)
tn.set_default_backend("tensorflow")

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 0-255の値が入っているので、0-1に収まるよう正規化します
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
print(x_train.shape, x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [4]:
x_vec = []
for i in range(x_train.shape[0]):
    x_vec_ = np.concatenate([x_train[i][j, ::-2*(j%2)+1] for j in range(x_train.shape[1])])
    x_vec.append(x_vec_)
x_train_1d = np.vstack(x_vec)

x_vec = []
for i in range(x_test.shape[0]):
    x_vec_ = np.concatenate([x_test[i][j, ::-2*(j%2)+1] for j in range(x_test.shape[1])])
    x_vec.append(x_vec_)
x_test_1d = np.vstack(x_vec)

In [5]:
# Retrieving a component

def block(*dimensions, norm = 1):
    '''Construct a new matrix for the MPS with random numbers from 0 to 1'''
    size = tuple([x for x in dimensions])
    return np.random.normal(loc = 0, scale = 0.34, size = size)

def create_MPS(rank, dim, bond_dim):
    '''Build the MPS tensor'''
    mps = [
        tn.Node( block(dim, bond_dim) )] + \
        [tn.Node( block(bond_dim, dim, bond_dim)) for _ in range(rank-2)] + \
        [tn.Node( block(bond_dim, dim) )
        ]

    #connect edges to build mps
    connected_edges=[]
    conn=mps[0][1]^mps[1][0]
    connected_edges.append(conn)
    for k in range(1,rank-1):
        conn=mps[k][2]^mps[k+1][0]
        connected_edges.append(conn)

    return mps, connected_edges

def create_MPS_labeled(rank, dim, bond_dim):
    '''Build the MPS tensor'''
    half = np.int((rank - 2) / 2)
    norm = 1 / bond_dim
    mps = [
        tn.Node( block(dim, bond_dim, norm=norm) )] + \
        [tn.Node( block(bond_dim, dim, bond_dim, norm = norm)) for _ in range(half)] + \
        [tn.Node( block(bond_dim, label_dim, bond_dim, norm=norm) )] + \
        [tn.Node( block(bond_dim, dim, bond_dim, norm=norm)) for _ in range(half, rank-2)] + \
        [tn.Node( block(bond_dim, dim, norm=norm) )
        ]

    #connect edges to build mps
    connected_edges=[]
    conn=mps[0][1]^mps[1][0]
    connected_edges.append(conn)
    for k in range(1,rank):
        conn=mps[k][2]^mps[k+1][0]
        connected_edges.append(conn)

    return mps, connected_edges


In [6]:
def feature_map(p):
    phi = [1-p, p]
    return phi

def data_tensorize(vec):
    data_tensor = [tn.Node(feature_map(p)) for p in vec]
    return data_tensor

In [7]:
label_len = 1
label_dim = 10
data_len = x_train_1d.shape[1]
rank = data_len
dim = 2
bond_dim = 10
# mps, edges = create_MPS(rank, dim, bond_dim)
mps, edges = create_MPS_labeled(rank, dim, bond_dim)

test_vec = x_train_1d[0]
data_tensor = data_tensorize(test_vec)

edges.append(data_tensor[0][0] ^ mps[0][0])
half_len = np.int(len(data_tensor) / 2)
[edges.append(data_tensor[i][0] ^ mps[i][1]) for i in range(1, half_len)]
[edges.append(data_tensor[i-label_len][0] ^ mps[i][1]) for i in range(half_len + label_len, data_len + label_len)]
for k in reversed(range(len(edges))):
    A = tn.contract(edges[k])
result = A.tensor

In [8]:
print(A.tensor.numpy())
print(A.tensor.numpy().astype("float32"))
print(tf.math.log(A.tensor).numpy())
print(tf.nn.softmax(A.tensor).numpy())

[-0.1258935  -0.18227812 -0.9654826  -0.68427782  0.12575317 -0.22343786
 -0.29122844 -0.96080808  0.03186352  0.83127648]
[-0.1258935  -0.18227813 -0.9654826  -0.68427783  0.12575316 -0.22343786
 -0.29122844 -0.9608081   0.03186351  0.8312765 ]
[        nan         nan         nan         nan -2.07343429         nan
         nan         nan -3.44629364 -0.18479284]
[0.09804769 0.09267228 0.04234562 0.05609629 0.1261032  0.08893534
 0.08310618 0.04254402 0.11480224 0.25534714]


In [9]:
def block(*dimensions):
    '''Construct a new matrix for the MPS with random numbers from 0 to 1'''
    size = tuple([x for x in dimensions])
    return tf.Variable(
        tf.random.normal(shape=size, dtype=tf.dtypes.float64, mean= 0, stddev = 0.34),
        trainable=True)

def create_blocks(rank, dim, bond_dim, label_dim):
    half = np.int((rank - 2) / 2)
    blocks = [
        block(dim, bond_dim) ] + \
        [ block(bond_dim, dim, bond_dim) for _ in range(half)] + \
        [ block(bond_dim, label_dim, bond_dim) ] + \
        [ block(bond_dim, dim, bond_dim) for _ in range(half, rank-2)] + \
        [ block(bond_dim, dim) 
        ]
    return blocks

def create_MPS_labeled(blocks, rank, dim, bond_dim):
    '''Build the MPS tensor'''
    half = np.int((rank - 2) / 2)
    mps = []
    for b in blocks:
        mps.append(tn.Node(b))

    #connect edges to build mps
    connected_edges=[]
    conn=mps[0][1]^mps[1][0]
    connected_edges.append(conn)
    for k in range(1,rank):
        conn=mps[k][2]^mps[k+1][0]
        connected_edges.append(conn)

    return mps, connected_edges

In [10]:
class TNLayer(tf.keras.layers.Layer):
    def __init__(self, input_len, label_num, bond_dim):
        self.label_len = 1
        self.label_dim = label_num
        self.rank = input_len
        self.dim = 2
        self.bond_dim = bond_dim
        #super(TNLayer, self).__init__()
        super().__init__()
        # Create the variables for the layer.
        self.blocks = create_blocks(self.rank, self.dim, self.bond_dim, self.label_dim)

    def call(self, inputs):
        def f(input_vec, blocks, rank, dim, bond_dim, label_len):
            mps, edges = create_MPS_labeled(blocks, rank, dim, bond_dim)
            data_tensor = []
            for p in tf.unstack(input_vec):
                data_tensor.append(tn.Node([1-p, p]))
            edges.append(data_tensor[0][0] ^ mps[0][0])
            half_len = np.int(rank / 2)
            [edges.append(data_tensor[i][0] ^ mps[i][1]) for i in range(1, half_len)]
            [edges.append(data_tensor[i-label_len][0] ^ mps[i][1]) \
                 for i in range(half_len + label_len, rank + label_len)]
            for k in reversed(range(len(edges))):
                A = tn.contract(edges[k])
            #result = tf.math.log(A.tensor)
            result = A.tensor - tf.math.reduce_max(A.tensor)
            return result

        result = tf.vectorized_map(
        lambda vec: f(vec, self.blocks, self.rank, self.dim, self.bond_dim, self.label_len), inputs)
        return result

In [11]:
N = x_train_1d.shape[1]
label_len = 1
label_num = 10
data_len = x_train_1d.shape[1]
rank = data_len
dim = 2
bond_dim = 10

tf.keras.backend.set_floatx('float64')

tn_model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(N,)),
        TNLayer(N, label_num, bond_dim),
        tf.keras.layers.Softmax()
    ])

tn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tn_layer (TNLayer)           (None, 10)                157440    
_________________________________________________________________
softmax (Softmax)            (None, 10)                0         
Total params: 157,440
Trainable params: 157,440
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def step_decay(epoch):
    x = 1e-5
    if epoch >= 100 and epoch <= 200:
        x = 1e-4
    return x

decay = LearningRateScheduler(step_decay, verbose=1)

optimizer = tf.keras.optimizers.Adam()
tn_model.compile(optimizer=optimizer, 
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                 metrics=['accuracy'])
batch = 32
tn_model.fit(x_train_1d, y_train, 
             batch_size=batch, epochs=300, 
             verbose=1, shuffle=True, 
             steps_per_epoch=int(60000 / batch), callbacks=[decay])


Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 1/300
1875/1875 [==============================] - 179s 95ms/step - loss: 624064.2729 - accuracy: 0.0991

Epoch 00002: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 2/300
1875/1875 [==============================] - 174s 93ms/step - loss: 79956.6907 - accuracy: 0.1021

Epoch 00003: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 3/300
1875/1875 [==============================] - 176s 94ms/step - loss: 1160.5092 - accuracy: 0.10290s - loss: 1163.8058 - accuracy:

Epoch 00004: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 4/300
1875/1875 [==============================] - 177s 95ms/step - loss: 649.7942 - accuracy: 0.1035

Epoch 00005: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 5/300
1875/1875 [==============================] - 166s 89ms/step - loss: 492.8239 - accuracy: 0.1045

Epoch 00006: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 6/

1875/1875 [==============================] - 166s 89ms/step - loss: 2.3003 - accuracy: 0.1268

Epoch 00048: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 48/300
1875/1875 [==============================] - 166s 89ms/step - loss: 2.2997 - accuracy: 0.1304

Epoch 00049: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 49/300
1875/1875 [==============================] - 165s 88ms/step - loss: 2.2983 - accuracy: 0.1321

Epoch 00050: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 50/300
1875/1875 [==============================] - 167s 89ms/step - loss: 2.2977 - accuracy: 0.1339

Epoch 00051: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 51/300
1875/1875 [==============================] - 163s 87ms/step - loss: 2.2990 - accuracy: 0.1344

Epoch 00052: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 52/300
1875/1875 [==============================] - 167s 89ms/step - loss: 2.2974 - accuracy: 0.1349

Epoch 00053: LearningRate

In [22]:
label_len = 1
label_num = 10
dim = 2
bond_dim = 5

tf.keras.backend.set_floatx('float64')

tn_model2 = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(28, 28, 1)),
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        TNLayer(196, label_num, bond_dim),
        tf.keras.layers.Softmax()
    ],name="sequential_1")

tn_model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
average_pooling2d_4 (Average (None, 14, 14, 1)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 196)               0         
_________________________________________________________________
tn_layer_4 (TNLayer)         (None, 10)                9970      
_________________________________________________________________
softmax_4 (Softmax)          (None, 10)                0         
Total params: 9,970
Trainable params: 9,970
Non-trainable params: 0
_________________________________________________________________


In [23]:
from tensorflow.keras.callbacks import LearningRateScheduler

In [24]:
x_train_2d = x_train[:, :, :, tf.newaxis]

def step_decay(epoch):
    x = 2e-5
    if epoch >= 100 and epoch <= 200:
        x = 2e-4
    return x

decay = LearningRateScheduler(step_decay, verbose=1)

optimizer = tf.keras.optimizers.Adam()
tn_model2.compile(optimizer=optimizer, 
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                 metrics=['accuracy'])
history = tn_model2.fit(x_train_2d, y_train, batch_size=32, epochs=300, verbose=1, callbacks=[decay])


Epoch 00001: LearningRateScheduler reducing learning rate to 2e-05.
Epoch 1/300
1875/1875 [==============================] - 50s 27ms/step - loss: 2.1696 - accuracy: 0.1044

Epoch 00002: LearningRateScheduler reducing learning rate to 2e-05.
Epoch 2/300
1875/1875 [==============================] - 49s 26ms/step - loss: 2.0331 - accuracy: 0.1045

Epoch 00003: LearningRateScheduler reducing learning rate to 2e-05.
Epoch 3/300
1875/1875 [==============================] - 48s 26ms/step - loss: 1.9991 - accuracy: 0.10450s

Epoch 00004: LearningRateScheduler reducing learning rate to 2e-05.
Epoch 4/300
1875/1875 [==============================] - 50s 27ms/step - loss: 1.9835 - accuracy: 0.1045

Epoch 00005: LearningRateScheduler reducing learning rate to 2e-05.
Epoch 5/300
1875/1875 [==============================] - 50s 27ms/step - loss: 1.9742 - accuracy: 0.1045

Epoch 00006: LearningRateScheduler reducing learning rate to 2e-05.
Epoch 6/300
1875/1875 [==============================] - 51

1875/1875 [==============================] - 51s 27ms/step - loss: 1.5549 - accuracy: 0.3805

Epoch 00095: LearningRateScheduler reducing learning rate to 2e-05.
Epoch 95/300
1875/1875 [==============================] - 50s 27ms/step - loss: 1.5528 - accuracy: 0.3802

Epoch 00096: LearningRateScheduler reducing learning rate to 2e-05.
Epoch 96/300
1875/1875 [==============================] - 49s 26ms/step - loss: 1.5508 - accuracy: 0.3816

Epoch 00097: LearningRateScheduler reducing learning rate to 2e-05.
Epoch 97/300
1875/1875 [==============================] - 50s 27ms/step - loss: 1.5489 - accuracy: 0.3814

Epoch 00098: LearningRateScheduler reducing learning rate to 2e-05.
Epoch 98/300
1875/1875 [==============================] - 51s 27ms/step - loss: 1.5469 - accuracy: 0.3817

Epoch 00099: LearningRateScheduler reducing learning rate to 2e-05.
Epoch 99/300
1875/1875 [==============================] - 50s 27ms/step - loss: 1.5456 - accuracy: 0.3816

Epoch 00100: LearningRateSchedu

1875/1875 [==============================] - 49s 26ms/step - loss: 0.9097 - accuracy: 0.7054

Epoch 00187: LearningRateScheduler reducing learning rate to 0.0002.
Epoch 187/300
1875/1875 [==============================] - 47s 25ms/step - loss: 0.9088 - accuracy: 0.7040

Epoch 00188: LearningRateScheduler reducing learning rate to 0.0002.
Epoch 188/300
1875/1875 [==============================] - 51s 27ms/step - loss: 0.9077 - accuracy: 0.7058

Epoch 00189: LearningRateScheduler reducing learning rate to 0.0002.
Epoch 189/300
1875/1875 [==============================] - 51s 27ms/step - loss: 0.9058 - accuracy: 0.7051

Epoch 00190: LearningRateScheduler reducing learning rate to 0.0002.
Epoch 190/300
1875/1875 [==============================] - 51s 27ms/step - loss: 0.9060 - accuracy: 0.7063

Epoch 00191: LearningRateScheduler reducing learning rate to 0.0002.
Epoch 191/300
1875/1875 [==============================] - 51s 27ms/step - loss: 0.9041 - accuracy: 0.7059

Epoch 00192: Learning

KeyboardInterrupt: 

In [16]:
x_test_2d = x_test[:, :, :, tf.newaxis]
pred = tn_model2.predict(x_test_2d)

In [19]:
pred[1]

array([9.88396434e-05, 5.02506551e-05, 2.41412798e-01, 1.84134023e-02,
       2.93382341e-05, 6.80038160e-02, 6.71760264e-01, 1.64887619e-07,
       1.26278551e-04, 1.04848077e-04])

In [20]:
test_loss, test_acc = tn_model2.evaluate(x_test_2d, y_test, verbose=1)

313/313 [==============================] - 8s 24ms/step - loss: 0.9037 - accuracy: 0.7687


In [26]:
from tensorflow.keras.models import load_model

tn_model2.save('./TN_MNIST_model_lr5em4_batch64.h5')
#del model
#model = load_model('/path/to/model.h5')

NotImplementedError: Layer TNLayer has arguments in `__init__` and therefore must override `get_config`.